# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [2]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('https://raw.githubusercontent.com/muriloavila/datascience/muriloavila-exercicio-modulo-5-1/SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape

# Não há duplicados

(27028, 69)


(27028, 69)

In [3]:
sinasc.head()


,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [4]:
# 2) 


sinasc.isna().sum()


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [5]:
# 3)

sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [6]:
# 4) 

# Gerando um backup da nossa variável para evitar danos aos dados originais:
backup_apgar5 = sinasc['APGAR5']

# Aplicando o método '.dropna()' para excluir as linhas com dados faltandos e
## aplicando o 'inplace = True' para que a ação do método prevaleça na versão original do backup:
backup_apgar5.dropna(inplace = True)


# Conferindo se conseguimos remover todos os valores nulos:
backup_apgar5.isna().sum() 

# O resultado "0", indica que não há mais valores nulos.

0

In [7]:
# 5) 

# Definindo o 9 como resultado "missing":
r = sinasc[['ESTCIVMAE', 'CONSULTAS']].replace({9: None})

# Conferindo se há algum valor como 9:
print(r.value_counts())
r


ESTCIVMAE  CONSULTAS
2.0        4            7686
1.0        4            5274
5.0        4            4985
1.0        3            2393
5.0        3            1723
2.0        3            1669
1.0        2             745
           1             513
5.0        2             353
4.0        4             337
2.0        1             319
           2             237
5.0        1             109
4.0        3              83
3.0        4              39
4.0        2              21
3.0        3              14
4.0        1              11
3.0        1               3
           2               1
dtype: int64


,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,None,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [8]:
# 6) 

# Usaremos o método '.fillna' para buscar os dados faltantes na coluna em questão e substituí-los por 0.
v = sinasc['QTDFILVIVO'].fillna(0)

# Checando se ainda há a existência de valores nulos dentro da variável que criámos:
v.isna().sum()

# O zero indica que não há mais dados faltantes, já que os mesmos foram substituídos por "0".

0

In [9]:
# 7) 
# Gerando uma variável que calcula qual a média de cada coluna:
media_das_colunas = sinasc.mean()

# Criando uma nova versão do nosso DataFrame onde os dados faltantes são substutuídos pelo valor médio
## da respectiva coluna:
sinasc_nona = sinasc.fillna(media_das_colunas)

# Realizando a confirmação de que no novo DataFrame não existe mais a ocorrência de dados faltantes:
sinasc_nona.isna().sum()

# O objetivo desta tomada de decisão foi garantir que os códigos gerados a partir deste
## DataFrame "sinasc_nona" possam seguir sendo executados de maneira mais limpa e leve com a 
### substituição dos dados faltantes, e que ainda seja possível ter análises precisas, visto que os 
### tais dados foram substituídos pela média, o que gera um impacto mínimo no resultado final.

C:\Users\jean\AppData\Local\Temp\ipykernel_9428\4165791057.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  media_das_colunas = sinasc.mean()


ORIGEM        0
CODESTAB      0
CODMUNNASC    0
LOCNASC       0
IDADEMAE      0
             ..
munResUf      0
munResLat     0
munResLon     0
munResAlt     0
munResArea    0
Length: 69, dtype: int64

In [10]:
# 8) 
# Categorizando os níveis de asfixia:
sinasc.loc[sinasc['APGAR5'] <= 10, 'ASFX_LVL'] = 'normal'
sinasc.loc[sinasc['APGAR5'] <= 7, 'ASFX_LVL'] = 'asfixia leve'
sinasc.loc[sinasc['APGAR5'] <= 5, 'ASFX_LVL'] = 'asfixia moderada'
sinasc.loc[sinasc['APGAR5'] <= 3, 'ASFX_LVL'] = 'asfixia severa'


# Gerando a frequência de exibição:

sinasc['ASFX_LVL'].value_counts()


normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: ASFX_LVL, dtype: int64

In [17]:
# 9) 
# Criando uma função responsável por alterar os caracteres para 'snake_case' e substituir os espaços
## por underscores:
def snake_case(text):
    return text.strip().lower().replace(' ', '_')

# Aplicando a função criada em nosso DataFrame, e alterando sua versão original com o Inplace = True:
sinasc.rename(columns=snake_case, inplace = True)

# Exibindo as colunas do DataFrame, vemos que todas estão em 'snake_case'.
sinasc.columns

Index(['origem', 'codestab', 'codmunnasc', 'locnasc', 'idademae', 'estcivmae',
       'escmae', 'codocupmae', 'qtdfilvivo', 'qtdfilmort', 'codmunres',
       'gestacao', 'gravidez', 'parto', 'consultas', 'dtnasc', 'horanasc',
       'sexo', 'apgar1', 'apgar5', 'racacor', 'peso', 'idanomal', 'dtcadastro',
       'codanomal', 'numerolote', 'versaosist', 'dtrecebim', 'difdata',
       'dtrecoriga', 'naturalmae', 'codmunnatu', 'codufnatu', 'escmae2010',
       'seriescmae', 'dtnascmae', 'racacormae', 'qtdgestant', 'qtdpartnor',
       'qtdpartces', 'idadepai', 'dtultmenst', 'semagestac', 'tpmetestim',
       'consprenat', 'mesprenat', 'tpapresent', 'sttrabpart', 'stcesparto',
       'tpnascassi', 'tpfuncresp', 'tpdocresp', 'dtdeclarac', 'escmaeagr1',
       'stdnepidem', 'stdnnova', 'codpaisres', 'tprobson', 'paridade',
       'kotelchuck', 'contador', 'munresstatus', 'munrestipo', 'munresnome',
       'munresuf', 'munreslat', 'munreslon', 'munresalt', 'munresarea',
       'asfx_lvl'],
   